In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Merging dataframes

First import all the different csv files as dataframes

In [2]:
auctions = pd.read_csv('tba_auctions_3_years.csv')
bids = pd.read_csv('tba_bids_3_years.csv')
buyer_set = pd.read_csv('tba_buyer_settings_3_years.csv')
buyers = pd.read_csv('tba_buyers_3_years.csv')
category = pd.read_csv('tba_category_3_years.csv')
lots = pd.read_csv('tba_lots_3_years.csv')

C:\Users\90538\anaconda3\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Create a big dataframe where the lots and auction dataframe are combined

In [3]:
merge_cat = pd.merge(category,lots,on='category_id',how='inner')
lots_auc = pd.merge(merge_cat,auctions,on='auction_id',how='inner')

Create a big dataframe for the bids and the buyer information

In [4]:
merge_buy = pd.merge(buyer_set,buyers,on='fake_buyer_id',how='inner')
buy_bid = pd.merge(merge_buy,bids,on='fake_buyer_id',how='inner')

This results in two big dataframes: lots_auc and buy_bid

In [ ]:
lots_auc.info()
len(lots_auc)

In [ ]:
buy_bid.info()

# Cleaning Data

We start with cleaning the data of the Lots_auc dataframe. This dataframe seems most important for our research topic: Auction performance.

First remove all the rows that have a NaN value for important columns

In [5]:
lots_auc = lots_auc.dropna(subset=['lot_quantity', 'estimated_price', 'nr_bids','highest_valid_bid_amount','nr_valid_bid_users','is_sold', 'auction_country','category_id'])

Since we have a dataset considering the data of 3-years we decided to delete the lots with an estimated price of 0 or less than 0. 

In [6]:
lots_auc.drop(lots_auc[lots_auc['estimated_price']<=0].index, inplace=True)

Than we delete all the lots where the starting price is higher than the estimated price

In [7]:
lots_auc = lots_auc[lots_auc['starting_price']<=lots_auc['estimated_price']]

Than we filtered the dataframe to discard data above the 99th percentile of lot quantity

In [8]:
percentile_99 = lots_auc['lot_quantity'].quantile(0.99)
lots_auc = lots_auc[lots_auc['lot_quantity']<= percentile_99]

# Feature Extraction

In [9]:
# Lot description length
lots_auc['lot_description_length'] = lots_auc['lot_description'].apply(lambda x: len(x) if pd.notna(x) else 0)

# Lot title length
lots_auc['lot_title_length'] = lots_auc['lot_title'].apply(lambda x: len(x) if pd.notna(x) else 0)


In [10]:
# Day and Month 

lots_auc['auction_start_time'] = pd.to_datetime(lots_auc['auction_start_time'], errors= 'coerce')
lots_auc['auction_start_time'] = lots_auc['auction_start_time'].dt.floor('S')

lots_auc['lot_close_time_actual'] = pd.to_datetime(lots_auc['lot_close_time_actual'])

# Add new columns for the day of the week and the month
lots_auc['auction_day_of_week'] = lots_auc['auction_start_time'].dt.day_name()
lots_auc['auction_month'] = lots_auc['auction_start_time'].dt.month_name()

In [ ]:
# Uniqueness

lots_auc['auction_start_time'] = lots_auc['auction_start_time'].dt.tz_localize(None)
lots_auc['lot_close_time_actual'] = lots_auc['lot_close_time_actual'].dt.tz_localize(None)

def calculate_uniqueness(row, df):
    # Filter the DataFrame to only those rows that are in the same category as the current row
    same_category = df[df['category_id'] == row['category_id']]
    
    # Filter to lots that are not closed and were opened before the starting time of the current auction
    open_lots = same_category[(same_category['lot_close_time_actual'] > row['auction_start_time']) &
                              (same_category['auction_start_time'] <= row['auction_start_time'])]
    
    # Return the count of these lots
    return open_lots.shape[0]

lots_auc['uniqueness'] = lots_auc.apply(lambda row: calculate_uniqueness(row, lots_auc), axis=1)
lots_auc['uniqueness']




In [ ]:
# TBA

# buy_bid['bid_datetime'] = pd.to_datetime(buy_bid['bid_datetime'], errors='coerce')

# buy_bid = buy_bid.sort_values(by=['fake_buyer_id', 'bid_datetime'])

import dask.dataframe as dd

# Convert your large pandas DataFrame to a Dask DataFrame
dask_df = dd.from_pandas(buy_bid, npartitions=10)

# Perform the sort
dask_df = dask_df.sort_values(by=['fake_buyer_id', 'bid_datetime'])
buy_bid = dask_df.compute()  # This converts it back to pandas DataFrame

# Calculate the time difference between consecutive bids for each bidder
buy_bid['time_diff'] = buy_bid.groupby('fake_buyer_id')['bid_datetime'].diff().dt.total_seconds()

tba_values = buy_bid.groupby('fake_buyer_id')['time_diff'].mean()

# Fill NaN values with M (meaning the bidder made only one bid)
tba_values = tba_values.fillna(10000)

tba_values = tba_values.reset_index()
tba_values.columns = ['fake_buyer_id', 'TBA']

# Merge with the original DataFrame
buy_bid = buy_bid.merge(tba_values, on='fake_buyer_id', how='left')
buy_bid['TBA']

In [18]:
pip install dask

Note: you may need to restart the kernel to use updated packages.


In [ ]:
highest_bids = lots_auc.groupby('lot_id')['highest_valid_bid_amount'].max().reset_index()
highest_bids.columns = ['lot_id', 'max_bid_amount']
highest_bids

In [ ]:
lots_auc[lots_auc['lot_id']=='00006e06-dd75-4ec6-89aa-c0ed28346bca']

In [ ]:
winning_bids

In [11]:
# Win Ratio

buy_bid = pd.merge(buy_bid, lots_auc[['lot_id','highest_valid_bid_amount']], how='left', on='lot_id')
buy_bid['is_won'] = buy_bid['highest_valid_bid_amount'] == buy_bid['amount_in_euro']



# Calculate total bids and won bids
bidder_stats = buy_bid.groupby('fake_buyer_id').agg(
    total_bids=pd.NamedAgg(column='id', aggfunc='count'),
    bids_won=pd.NamedAgg(column='is_won', aggfunc='sum')
)

# Calculate the win ratio
bidder_stats['win_ratio'] = bidder_stats['bids_won'] / bidder_stats['total_bids']

# Reset index to merge
bidder_stats = bidder_stats.reset_index()

# Merge the win ratios back to the original DataFrame
buy_bid = buy_bid.merge(bidder_stats[['fake_buyer_id', 'win_ratio']], on='fake_buyer_id', how='left')


In [13]:
buy_bid['fake_buyer_id'].head()

0    1
1    1
2    1
3    1
4    1
Name: fake_buyer_id, dtype: int64

In [15]:
buy_bid[buy_bid['fake_buyer_id']==1].head()

MemoryError: Unable to allocate 1.69 GiB for an array with shape (9, 25265755) and data type object